In [1]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import streamlit as st
import missingno as msno
import os
import requests
from bs4 import BeautifulSoup
import random

import sqlite3
from sqlalchemy import create_engine, MetaData, Table

from helper import *


pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

In [2]:
#Loading merged dataset

title_basics = pd.read_csv('../data/title_basics_data.tsv', sep = '\t', low_memory=False)
title_basics.replace('\\N', np.nan, inplace=True)
title_basics = title_basics.dropna(subset=['startYear'])
title_basics['startYear'] = title_basics['startYear'].astype('int')
title_basics = title_basics[(title_basics['titleType']=='movie') & (title_basics['startYear']>2013)]

print(title_basics.shape, title_basics['tconst'].nunique())

(170952, 9) 170952


In [3]:
proxy_list = pd.read_csv('./Free_Proxy_List.csv')

proxy_list['full']=proxy_list['ip'].astype('string')+':'+proxy_list['port'].astype('string')
proxies = proxy_list['full'].tolist()

In [4]:
# Check if the progress file exists
if os.path.exists("progress.txt"):
    with open("progress.txt", "r") as file:
        start_index = int(file.read())
else:
    start_index = 0

# Check if the revenue data file exists
if os.path.exists("df_revenue.csv"):
    df_revenue = pd.read_csv("df_revenue.csv")
    worldwide_revenue = df_revenue["worldwide_revenue"].tolist()
    films = df_revenue["tconst"].tolist()
else:
    worldwide_revenue = []
    films = []

# Scraping data from boxofficemojo.com
new_films = title_basics['tconst'].unique()[start_index:]

break_counter = 0

for i, film in enumerate(new_films, start=start_index):
    print("=" * 80)
    if break_counter == 10:
        print("More than 10 bad requests")
        break

    proxy = random.choice(proxies)
    revenue = get_worldwide_revenue(film, proxy)
    worldwide_revenue.append(revenue)
    films.append(film)

    # Save data after processing 5 films
    if i % 5 == 0:
        with open("progress.txt", "w") as file:
            file.write(str(i + 1))
        df_revenue = pd.DataFrame({"tconst": films, "worldwide_revenue": worldwide_revenue})
        df_revenue.to_csv("df_revenue.csv", index=False)
        print("df_revenue updated")

# Remove the progress file as the processing is complete
os.remove("progress.txt")


Proxy 101.51.121.203:4153 successfully connected
No worldwide revenue info provided for tt9070818
Proxy 186.251.255.141:31337 successfully connected
No worldwide revenue info provided for tt9070824
Proxy 45.8.107.79:80 successfully connected
Worldwide Revenue for tt9070934: $29973
df_revenue updated
Proxy 159.223.163.20:44171 successfully connected
No worldwide revenue info provided for tt9071014
Proxy 78.90.19.79:63246 successfully connected
No worldwide revenue info provided for tt9071040
Proxy 96.70.52.227:48324 successfully connected
Worldwide Revenue for tt9071066: $75381
Proxy 58.240.192.158:4145 successfully connected
No worldwide revenue info provided for tt9071068
Proxy 155.133.83.161:58351 successfully connected
No worldwide revenue info provided for tt9071108
df_revenue updated
Proxy 139.180.223.81:26171 successfully connected
No worldwide revenue info provided for tt9071144
Proxy 8.42.68.109:39593 successfully connected
No worldwide revenue info provided for tt9071172
Proxy